<a href="https://colab.research.google.com/github/JoaoVitorSampaio/OperationalResearch_UFPB/blob/main/Pr%C3%A1tica_PLI_BinPacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Bin Packing


## Exercício 1


### Descrição do problema
Uma faculdade quer agendar as provas de 6 disciplinas. Existe uma regra que proíbe que o mesmo aluno tenha que fazer mais de uma prova por dia. A tabela abaixo mostra quais alunos (identificados por números) vão fazer a prova de cada disciplina. Por exemplo, A e B não podem ser agendadas no mesmo dia por causa do aluno 1. Já B e C poderiam ser agendadas no mesmo dia, pois nenhum aluno vai fazer essas duas provas.

| Disciplina | Alunos |
|:---:|:---:|
| A | 1, 2, 3, 4 |
| B | 1, 5 |
| C | 3, 6, 7 |
| D | 5, 7 |
| E | 2, 7 |
| F | 4, 5 |

Crie um modelo de PLI para agendar essas provas no menor número de dias possível.
**Dica: baseie-se no modelo do problema de empacotamento de caixas.**


### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Dados
$A = \{1, 2, 3, 4, 5, 6, 7\}$: conjunto de alunos \\
$D = \{1, 2, 3, 4, 5, 6\}$: conjunto de dias em que é possível alocar provas \\
$M = \{A, B, C, D, E, F\}$: conjunto de disciplinas \\
$M_a$: conjunto de disciplinas que o aluno $a$ irá fazer prova \\

In [ ]:
A = range(1, 8)
D = range(1, 7)
M = ['A', 'B', 'C', 'D', 'E', 'F']
Ma = {1: ['A', 'B'], 2: ['A', 'E'], 3: ['A', 'C'], 4: ['A', 'F'], 5: ['B', 'D', 'F'], 6: ['C'], 7: ['C', 'D', 'E']}


#### Variável
$x_{dm}$ = 1 se a prova da disciplina $m$ acontecerá no dia $d$, 0 c.c.  \\
$y_d$ = 1 se haverá alguma prova no dia $d$, 0 c.c.

#### Modelo

$$\min \sum_{d \in D} y_d$$
s.t.
$$\sum_{d \in D} x_{dm} = 1; \forall m \in M$$
$$\sum_{m \in M_a} x_{dm} \leq y_d ; \forall d \in D; \forall a \in A$$
$$x \text{ e } y \text{ binários}$$


In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = {d: {m: model.add_var(var_type=BINARY, name=f"x_{d}_{m}") for m in M} for d in D}
y = {d: model.add_var(var_type=BINARY, name=f"y_{d}") for d in D}

model.objective = xsum(y[d] for d in D)

for m in M:
    model += xsum(x[d][m] for d in D) == 1

for d in D:
  for a in A:
    model += xsum(x[d][m] for m in Ma[a]) <= y[d]

# salva modelo em arquivo lp, e mostra o conteúdo
model.write("model.lp") # salva modelo em arquivo
with open("model.lp", "r") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: y_1 + y_2 + y_3 + y_4 + y_5 + y_6
Subject To
constr(0):  x_1_A + x_2_A + x_3_A + x_4_A + x_5_A + x_6_A = 1
constr(1):  x_1_B + x_2_B + x_3_B + x_4_B + x_5_B + x_6_B = 1
constr(2):  x_1_C + x_2_C + x_3_C + x_4_C + x_5_C + x_6_C = 1
constr(3):  x_1_D + x_2_D + x_3_D + x_4_D + x_5_D + x_6_D = 1
constr(4):  x_1_E + x_2_E + x_3_E + x_4_E + x_5_E + x_6_E = 1
constr(5):  x_1_F + x_2_F + x_3_F + x_4_F + x_5_F + x_6_F = 1
constr(6):  x_1_A + x_1_B - y_1 <= -0
constr(7):  x_1_A + x_1_E - y_1 <= -0
constr(8):  x_1_A + x_1_C - y_1 <= -0
constr(9):  x_1_A + x_1_F - y_1 <= -0
constr(10):  x_1_B + x_1_D + x_1_F - y_1 <= -0
constr(11):  x_1_C - y_1 <= -0
constr(12):  x_1_C + x_1_D + x_1_E - y_1 <= -0
constr(13):  x_2_A + x_2_B - y_2 <= -0
constr(14):  x_2_A + x_2_E - y_2 <= -0
constr(15):  x_2_A + x_2_C - y_2 <= -0
constr(16):  x_2_A + x_2_F - y_2 <= -0
constr(17):  x_2_B + x_2_D + x_2_F - y_2 <= -0
constr(18):  x_2_C - y_2 <= -0
constr(19):  x_2_C + x_2_D + x_2_E - y

In [ ]:
status = model.optimize()

print(f"Status = {status}\n")
if status == OptimizationStatus.OPTIMAL:
  print(f"São necessários no mínimo {int(model.objective_value)} dias para realizar as provas.\n")

  day = 1
  for d in D:
    if int(y[d].x) == 1:
      print(f"Dia {day}: ", ", ".join([m for m in M if int(x[d][m].x) == 1]))
      day += 1

Status = OptimizationStatus.OPTIMAL

São necessários no mínimo 3 dias para realizar as provas.

Dia 1:  E, F
Dia 2:  A, D
Dia 3:  B, C
